In [9]:
import pandas as pd
import numpy as np
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk import sent_tokenize,word_tokenize 
from nltk.book import *
import gensim.summarization

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [3]:
all_data = pd.read_pickle('../../Cleaned Data/italian_review_551.pkl')
sample = all_data['review'].iloc[0]

In [39]:
#Divide into different clusters
cluster0 = all_data[all_data['label']==0]
cluster1 = all_data[all_data['label']==1]
cluster2 = all_data[all_data['label']==2]
cluster3 = all_data[all_data['label']==3]
cluster4 = all_data[all_data['label']==4]

In [50]:
def get_text(cluster):
    text_list = ''
    for item in cluster['review']:
        text_list += str(item)
    return text_list

In [51]:
text_list0 = get_text(cluster0)
text_list1 = get_text(cluster1)
text_list2 = get_text(cluster2)
text_list3 = get_text(cluster3)
text_list4 = get_text(cluster4)

### Data Cleaning and Topic Analysis

In [100]:
from operator import itemgetter
# 'Good' and 'great' are widely used in comments, because the common comments of resaurants are approximately positive. 
# So we delete 'good' and 'great'. And other words make no sense, we also delete them
Useless_words = ['will','one','great','good','-','really','italian']
def LDA_topic(sample,num,num_topics = 5,passes = 10,num_words=8,Useless_words=Useless_words):
    sents = sent_tokenize(sample)
    for index in range(len(sents)):
        sents[index] = sents[index].strip().replace('\n','').replace("\'","'")
    sentence = '.'.join(sents)
    #convert the whole text into words
    text = [word for word in sentence.lower().split() if word not in STOPWORDS and word.isalnum and word not in Useless_words]
    
    #get the unique token in the texts
    dictionary = corpora.Dictionary([text])
    #(word_id,freq) pairs by sentence,and generate index #bags of words
    corpus = [dictionary.doc2bow(txt) for txt in [text]] 
    
    #Set parameters
    num_topics = num_topics #The number of topics that should be generated
    passes = passes # The num of passes to refine the model
    lda = LdaModel(corpus, #after dealing with frequencies
                  id2word=dictionary,#id to words
                  num_topics=num_topics,#how many topics wanted
                  passes=passes) #passes search through
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(lda.print_topics(num_words=num_words))#each topic can assoicate with more than a word #return 8 words
    
    list_ = sorted(lda.get_document_topics(corpus[0],minimum_probability=0,per_word_topics=False),key=itemgetter(1),reverse=True)
    print('For cluster{},the topic is'.format(num),lda.print_topic(topicno=list_[0][0]),'\n')
    
    return lda
    
#     return lda.print_topics(num_words=num_words)

In [101]:
text_list = [text_list0,text_list1,text_list2,text_list3,text_list4]
for index in range(len(text_list)):
    LDA_topic(sample = text_list[index],num = index)

[   (   0,
        '0.000*"place" + 0.000*"food" + 0.000*"pizza" + 0.000*"service" + '
        '0.000*"pasta" + 0.000*"ordered" + 0.000*"back" + 0.000*"us"'),
    (   1,
        '0.001*"pizza" + 0.001*"food" + 0.001*"place" + 0.001*"us" + '
        '0.001*"restaurant" + 0.000*"pasta" + 0.000*"service" + 0.000*"back"'),
    (   2,
        '0.008*"food" + 0.008*"place" + 0.007*"pizza" + 0.006*"pasta" + '
        '0.005*"restaurant" + 0.005*"ordered" + 0.005*"service" + '
        '0.004*"back"'),
    (   3,
        '0.001*"pizza" + 0.000*"place" + 0.000*"food" + 0.000*"restaurant" + '
        '0.000*"pasta" + 0.000*"ordered" + 0.000*"back" + 0.000*"service"'),
    (   4,
        '0.001*"food" + 0.000*"pasta" + 0.000*"place" + 0.000*"pizza" + '
        '0.000*"ordered" + 0.000*"restaurant" + 0.000*"service" + '
        '0.000*"back"')]
For cluster0,the topic is 0.008*"food" + 0.008*"place" + 0.007*"pizza" + 0.006*"pasta" + 0.005*"restaurant" + 0.005*"ordered" + 0.005*"service" + 0.004*"bac

### Sentimental Analysis

In [119]:
word_batch = ['place','food','service','pizza','pasta','restaurant']
def get_sent(text_list,word_batch=word_batch):
    word_list = []
    text1 = sent_tokenize(text_list)
    for word in word_batch:
        for sent in text1:
            if word in sent:
                word_list.append(sent)
    word_set = set(word_list)
    word_str = ' '.join(list(word_set))
    return word_str


In [120]:
text_str0 = get_sent(text_list0)
text_str1 = get_sent(text_list1)
text_str2 = get_sent(text_list2)
text_str3 = get_sent(text_list3)
text_str4 = get_sent(text_list4)

In [118]:
def get_pos_neg_words():
    def get_words(url):
        import requests
        words = requests.get(url).content.decode('latin-1')
        word_list = words.split('\n')
        index = 0
        while index < len(word_list): #do cleaning
            word = word_list[index]
            if ';' in word or not word:
                word_list.pop(index)
            else:
                index+=1
        return word_list

    #Get lists of positive and negative words
    p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt' #list of positive words
    n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
    positive_words = get_words(p_url)#clean the words
    negative_words = get_words(n_url)
    return positive_words,negative_words

positive_words,negative_words = get_pos_neg_words()

In [121]:
def do_pos_neg_sentiment_analysis(text_list,debug=False):
    positive_words,negative_words = get_pos_neg_words()
    from nltk import word_tokenize
    results = list()
    for text in text_list:
        cpos = cneg = lpos = lneg = 0
        for word in word_tokenize(text[1]):
            if word in positive_words:
                if debug: 
                    print("Positive",word)
                cpos+=1
            if word in negative_words:
                if debug:
                    print("Negative",word)
                cneg+=1
        results.append((text[0],cpos/len(word_tokenize(text[1])),cneg/len(word_tokenize(text[1])))) #normlaized by length
    return results

pos_neg = do_pos_neg_sentiment_analysis([('cluster0',text_str0),('cluster1',text_str1),('cluster2',text_str2),('cluster3',text_str3),('cluster4',text_str4)])
pos_neg

[('cluster0', 0.05572068457594491, 0.013949948954413089),
 ('cluster1', 0.057921835081240575, 0.014208962375938518),
 ('cluster2', 0.058058923857792155, 0.014634756893784993),
 ('cluster3', 0.05686397160438154, 0.01420574392367884),
 ('cluster4', 0.05532745955303047, 0.01613440156239597)]

### Word Clouds

In [ ]:
texts = [('cluster0',text_list0),('cluster1',text_list1),('cluster2',text_list2),('cluster3',text_list3),('cluster4',text_list4)]
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
#Remove unwanted words
#As we look at the cloud, we can get rid of words that don't make sense by adding them to this variable
DELETE_WORDS = ['place','food','service','pizza','pasta','restaurant']
def remove_words(text_string,DELETE_WORDS=DELETE_WORDS):
    for word in DELETE_WORDS:
        text_string = text_string.replace(word,' ') #delelte the words, replace with none
    return text_string

#Remove short words
MIN_LENGTH = 3
def remove_short_words(text_string,min_length = MIN_LENGTH):
    word_list = text_string.split()
    for word in word_list:
        if len(word) < min_length:
            text_string = text_string.replace(' '+word+' ',' ',1) #if less than the min length then replace the word
    return text_string


#Set up side by side clouds
COL_NUM = 2
ROW_NUM = 2
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(12,12))

for i in range(0,len(texts)):
    text_string = remove_words(texts[i][1])
    text_string = remove_short_words(text_string)
    ax = axes[i//2, i%2] 
    ax.set_title(texts[i][0])
    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=1200,height=1000,max_words=20).generate(text_string)
    ax.imshow(wordcloud)
    ax.axis('off')
plt.show()